In [152]:
import pandas as pd
import plotly.express as px
import aqi as aqi
import matplotlib.pyplot as plt
import ruptures as rpt

In [160]:
# read in data
data_file = "waqi-covid19-airqualitydata-2020.csv"
df_2020 = pd.read_csv(data_file)

df_2019Q4 = pd.read_csv("waqi-covid19-airqualitydata-2019Q4.csv")
df_2019Q3 = pd.read_csv("waqi-covid19-airqualitydata-2019Q3.csv")
df_2019Q2 = pd.read_csv("waqi-covid19-airqualitydata-2019Q2.csv")
df_2019Q1 = pd.read_csv("waqi-covid19-airqualitydata-2019Q1.csv")

# df_2019 = df_2019Q4.append(df_2019Q3)

df_2019_1 = df_2019Q4.append(df_2019Q3)
df_2019_2 = df_2019_1.append(df_2019Q2)
df_2019 = df_2019_2.append(df_2019Q1)

df_2020['Date'] = df_2020['Date'].astype('datetime64[ns]')
df_2019['Date'] = df_2019['Date'].astype('datetime64[ns]')

bad_species = ['wind gust', 'temperature', 'wind speed', 'wind-gust', 'pressure', 'dew', 'wind-speed',
       'humidity', 'precipitation', 'wd']
# good_species = ['pm25', 'pm10', 'no2', 'o3', 'so2', 'co']

df_2020 = df_2020[~df_2020.Specie.isin(bad_species)]
df_2019 = df_2019[~df_2019.Specie.isin(bad_species)]

# df_2020 = df_2020[df_2020.Specie.isin(good_species)]
# df_2019 = df_2019[df_2019.Specie.isin(good_species)]

df_combined = df_2020.append(df_2019)
df_combined.shape

(1310177, 10)

In [161]:
df_combined.head()

,City,Country,Date,Specie,Unnamed: 0,count,max,median,min,variance
0,Dublin,IE,2020-01-03,o3,0.0,26,57.7,49.2,24.5,810.59
1,Dublin,IE,2020-01-09,o3,1.0,19,31.1,13.4,2.4,606.87
2,Dublin,IE,2020-01-10,o3,2.0,24,30.2,22.4,4.8,479.78
3,Dublin,IE,2020-02-10,o3,3.0,42,33.5,29.8,23.2,69.84
4,Dublin,IE,2020-02-11,o3,4.0,39,32.8,29.7,18.8,99.06


In [162]:
df_combined['Country'].unique()

array(['IE', 'IQ', 'KW', 'SG', 'AE', 'CW', 'GR', 'CY', 'DE', 'HK', 'NZ',
       'PL', 'RE', 'RO', 'BH', 'IN', 'JP', 'IR', 'MK', 'SK', 'ZA', 'BE',
       'BG', 'BO', 'PK', 'TJ', 'TR', 'VN', 'CH', 'LT', 'MN', 'AU', 'ID',
       'KG', 'PR', 'SV', 'GN', 'GT', 'NL', 'IS', 'LK', 'NO', 'RS', 'SA',
       'DK', 'DZ', 'HR', 'TM', 'ML', 'MM', 'US', 'CN', 'CO', 'ET', 'JO',
       'PT', 'SE', 'UZ', 'AT', 'CI', 'GH', 'EC', 'EE', 'MO', 'MX', 'PH',
       'BA', 'BD', 'CL', 'FI', 'FR', 'KR', 'ES', 'IT', 'LA', 'MY', 'AF',
       'BR', 'CZ', 'RU', 'UG', 'CR', 'IL', 'NP', 'PE', 'TH', 'UA', 'XK',
       'AR', 'CA', 'KZ', 'TW', 'GB', 'GE', 'HU'], dtype=object)

In [171]:
important_countries = ['US', 'SP', 'IT', 'FR', 'DE', 
                       'GB', 'TR', 'IR', 'CN', 'RU', 
                       'BR', 'BE', 'CA', 'NL', 'CH', 
                       'PT', 'IN', 'PE', 'IE', 'SE', 
                       'AT', 'IL', 'SA', 'JP', 'CL',
                       'KR', 'EC', 'PL', 'PK', 'MX']
df_combined =  df_combined[df_combined.Country.isin(important_countries)]
df_combined.shape

(951814, 10)

In [166]:
# df_combined.to_csv("airqualitydata-2019-2020.csv")

# Change Point

In [173]:
df_changepoints = None

In [190]:
df_combined[df_combined['Country']=="CN"]['City'].unique()

array(['Beijing', 'Jieyang', 'Kunming', 'Hangzhou', 'Chongqing',
       'Qingdao', 'Haikou', 'Ürümqi', 'Qiqihar', 'Guiyang', 'Shenzhen',
       'Yunfu', 'Xuchang', 'Yinchuan', 'Shenyang', 'Lhasa', 'Shanghai',
       'Changchun', 'Foshan', 'Nanning', 'Fushun', 'Hefei', 'Chengdu',
       'Hohhot', 'Qinhuangdao', 'Shijiazhuang', 'Shantou', 'Zhengzhou',
       'Nanjing', 'Xining', 'Xi’an', 'Zhuzhou', 'Wuhan', 'Tianjin',
       'Changzhou', 'Nanchang', 'Shiyan', 'Xinxiang', 'Suzhou', 'Harbin',
       'Lanzhou', 'Jinan', 'Changsha', 'Hegang', 'Anyang', 'Wuxi',
       'Taiyuan', 'Guangzhou', 'Fuzhou', 'Ningbo', 'Xiamen', 'Dongguan'],
      dtype=object)

In [175]:
unique_cities = df_combined['City'].unique()
unique_species = df_combined['Specie'].unique()

In [155]:
df_test = df_combined[(df_combined['City']=='Beijing') & (df_combined['Specie'] == 'no2')].sort_values(by=['Date'])

In [199]:
for c in unique_cities:
    print(c)
    df_city = df_combined[df_combined['City']==c]
    city_species = df_city['Specie'].unique()
    for s in unique_species:
        if s in city_species:
            try:
                df_test = df_city[(df_city['Specie'] == s)].reset_index(drop=True).sort_values(by=['Date'])
                signal = df_test['median'].to_numpy()
                algo = rpt.Pelt(model="rbf").fit(signal)
                result = algo.predict(pen=10)
                df_res= df_test.iloc[result[:-1],]
                if df_changepoints is not None:
                    df_changepoints = df_changepoints.append(df_res)
                else:
                    df_changepoints = df_res
            except Exception as e:
                print(e, c,s)
        

Dublin
Stuttgart
Münster
Köln
Kassel
Karlsruhe
Mainz
Dresden
Munich
Berlin
Düsseldorf
Freiburg
Wiesbaden
Hannover
Augsburg
Darmstadt
Potsdam
Hamburg
Łódź
Tarnów
Gdańsk
Katowice
Poznań
Rybnik
Szczecin
Kielce
Warsaw
Bydgoszcz
Zabrze
Wrocław
Płock
Kraków
Thrissur
New Delhi
Hyderabad
Delhi
Chandigarh
Bhopal
Nagpur
Lucknow
Ghāziābād
Hāpur
Gandhinagar
Chennai
Nashik
Mysore
Visakhapatnam
Mumbai
Jaipur
Patna
Muzaffarnagar
Thiruvananthapuram
Bengaluru
Shillong
Kolkata
Fukuoka
Kanazawa
Kumamoto
Akita
Kochi
Gifu-shi
Kyoto
Hiroshima
Shizuoka
Chiba
Sapporo
Yokohama
Tokyo
Saitama
Naha
Nagasaki
Toyama
Niigata
Nara-shi
Okayama
Sendai
Osaka
Miyazaki
Matsuyama
Wakayama
Takamatsu
Nagoya
Ōita
Kobe
Nagano
Utsunomiya
Kagoshima
Isfahan
Arāk
Karaj
Qom
Orūmīyeh
Yazd
Īlām
Kerman
Khorramshahr
Tabriz
Bandar Abbas
Sanandaj
Kermanshah
Khorramabad
Shiraz
Zanjān
Mashhad
Tehran
Liège
Brussels
Charleroi
Gent
Namur
Antwerpen
Karachi
Peshawar
Islamabad
Lahore
Eskişehir
İzmit
Bursa
Ankara
Adana
Kayseri
Kütahya
Balıkesir
A

In [212]:
df_changepoints = df_changepoints.reset_index(drop=True)
df_changepoints.head()

,City,Country,Date,Specie,Unnamed: 0,count,max,median,min,variance
0,Dublin,IE,2019-07-01,o3,NaN,31,23.4,19.1,11.9,110.53
1,Dublin,IE,2019-11-30,o3,NaN,15,27.2,25.0,23.3,12.61
2,Dublin,IE,2019-07-01,o3,NaN,31,23.4,19.1,11.9,110.53
3,Dublin,IE,2019-11-30,o3,NaN,15,27.2,25.0,23.3,12.61
4,Dublin,IE,2019-07-27,pm25,NaN,20,97.0,92.0,83.0,244.63


In [213]:
df_final = df_changepoints[df_changepoints['Date']>'2020']
df_final.shape

(1242, 10)

In [215]:
df_final=df_final[['City','Country','Date','Specie']]
df_final = df_final.drop_duplicates()
df_final

,City,Country,Date,Specie
10,Stuttgart,DE,2020-01-26,o3
41,Münster,DE,2020-01-27,o3
44,Köln,DE,2020-01-26,o3
47,Kassel,DE,2020-01-29,o3
110,Karlsruhe,DE,2020-01-26,o3
...,...,...,...,...
5799,Swansea,GB,2020-01-27,o3
5802,Swansea,GB,2020-02-01,no2
5804,Swansea,GB,2020-01-10,so2
5808,Swansea,GB,2020-01-14,co


In [222]:
# df_final.to_csv("changepoints.csv")
list(df_final[(df_final['City']=="Manhattan")&(df_final['Specie']=="o3")]['Date'].apply(str))

['2020-02-25 00:00:00']

In [227]:
df_changepoint = df_final
city = "Abha"
specie="co"
list(df_changepoint[(df_changepoint['City']==city)&(df_changepoint['Specie']==specie)]['Date'].apply(str))


[]

# AQI

In [167]:
df = df_combined

In [168]:
df.shape

(951814, 10)

In [169]:
# for purpose of using aqi package
d = {}
d['pm25'] = aqi.POLLUTANT_PM25
d['pm10'] = aqi.POLLUTANT_PM10
d['no2'] = aqi.POLLUTANT_NO2_1H
d['o3'] = aqi.POLLUTANT_O3_8H
d['so2'] = aqi.POLLUTANT_SO2_1H
d['co'] = aqi.POLLUTANT_CO_8H

df = df.drop(df[(df['Specie'] == 'co') & (df['median'] == 58.0)].index)

In [170]:
def compute_iaqi(row):
    if(row['Specie'] == 'o3'):
        row['median']/= 1000
    return aqi.to_iaqi(d[row['Specie']], row['median'], algo=aqi.ALGO_EPA)
df['iaqi'] = df.apply(compute_iaqi,axis=1)

IndexError: ('list index out of range', 'occurred at index 72064')

In [ ]:
# take max to get final AQI 
df.sort_values(by=['City','Date'])

df_new = df.drop(['Country','Specie','count','min','max','median','variance'], axis = 1)
df_new = df_new.groupby(['Date','City']).max()

print(type(df_new.axes[1]))
def AQI(row):
    #print(df_new.loc[(row['Date'],row['City'])]["iaqi"])
    return df_new.loc[(row['Date'],row['City'])]["iaqi"]

df['AQI'] = df.apply(AQI,axis=1)

In [ ]:
df = df.sort_values(by=['City','Date'])
print(df['City'].unique())

In [ ]:
### City -> County 

d_tocounty = {}
d_tocounty['Albuquerque'] = 'Bernalillo'
d_tocounty['Atlanta'] = 'Fulton'

In [ ]:
# COVID data: confirmed 
df_confirmed = pd.read_csv("time_series_covid19_confirmed_US.csv")
df_confirmed.head()

In [ ]:
df_confirmed = df_confirmed.drop(['UID','iso2','iso3','code3','FIPS'],axis=1)
df_confirmed = df_confirmed.dropna()